In [3]:
import requests
from bs4 import BeautifulSoup


In [4]:
url = 'https://finance.yahoo.com/markets/stocks/most-active/'
response = requests.get(url)


In [5]:
if response.status_code == 200:
    print("Request was successful!")
else:
    print(f"Request failed with status code: {response.status_code}")

Request was successful!


In [6]:
content=response.text

In [7]:
soup = BeautifulSoup(content, 'lxml')


In [8]:
company_name=[]
span_values = soup.find_all('span', class_='symbol yf-ravs5v')
print(span_values)

[<span class="symbol yf-ravs5v">NVDA</span>, <span class="symbol yf-ravs5v">NIO</span>, <span class="symbol yf-ravs5v">INTC</span>, <span class="symbol yf-ravs5v">TSLA</span>, <span class="symbol yf-ravs5v">PLTR</span>, <span class="symbol yf-ravs5v">BBD</span>, <span class="symbol yf-ravs5v">VALE</span>, <span class="symbol yf-ravs5v">SOFI</span>, <span class="symbol yf-ravs5v">BABA</span>, <span class="symbol yf-ravs5v">JD</span>, <span class="symbol yf-ravs5v">AMZN</span>, <span class="symbol yf-ravs5v">MARA</span>, <span class="symbol yf-ravs5v">F</span>, <span class="symbol yf-ravs5v">WULF</span>, <span class="symbol yf-ravs5v">RIVN</span>, <span class="symbol yf-ravs5v">PDD</span>, <span class="symbol yf-ravs5v">AAL</span>, <span class="symbol yf-ravs5v">TME</span>, <span class="symbol yf-ravs5v">SMAR</span>, <span class="symbol yf-ravs5v">ABEV</span>, <span class="symbol yf-ravs5v">CCL</span>, <span class="symbol yf-ravs5v">IQ</span>, <span class="symbol yf-ravs5v">U</span>, <sp

In [9]:
for i in span_values[:10]:
    title = i.get_text()

    company_name.append(title)



In [10]:
company_name

['NVDA', 'NIO', 'INTC', 'TSLA', 'PLTR', 'BBD', 'VALE', 'SOFI', 'BABA', 'JD']

In [11]:
import pandas as pd

In [12]:

def most_young_ceo():
    details=[]
    for k in company_name:
        profile_page = f'https://finance.yahoo.com/quote/{k}/profile/'
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
        }
        response = requests.get(profile_page, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        country_parent = soup.find('div', class_="address yf-wxp4ja")
        if country_parent:
            divs = country_parent.find_all('div')
        country = divs[-1].text.strip()
        Employees = soup.find('strong').text.strip()
        count = 0
        table = soup.find('table')
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            CEO_Name = cols[0].text.strip()
            Name = cols[1].text.strip()
            CEO_Year_Born = cols[4].text.strip()
                        # volume = cols[6].text.strip()
            if count==0:
                details.append({
                        "Name": Name,
                        "Country": country,
                        "Employees": Employees,
                        "CEO_Name": CEO_Name,
                        "CEO_Year_Born": CEO_Year_Born,
                        })
                count+=1
            else:
                break
        # print(row)
    return details
def tabulate():
    details = most_young_ceo()
    df = pd.DataFrame(details)
    sorted_df = df.sort_values("CEO_Year_Born", ascending=False)
    return sorted_df[:5]


tabulate()

,Name,Country,Employees,CEO_Name,CEO_Year_Born
1,"Co-Founder, Chairman & CEO",China,"32,820",Mr. William Li,1974
9,Founder & Chairman,China,"517,124",Mr. Qiangdong Liu,1973
3,"Co-Founder, Technoking of Tesla, CEO & Director",United States,"140,473",Mr. Elon R. Musk,1972
4,Co-Founder & Chairman,United States,"3,661",Mr. Peter Andreas Thiel,1967
5,CEO & Member of Executive Board,Brazil,"84,711",Mr. Marcelo de Araujo Noronha,1965


## 2. 10 stocks with best 52-Week Change. 52-Week Change placed on Statistics tab.
    Sheet's fields: Name, Code, 52-Week Change, Total Cash


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
details1 = []
def scrap_page():
    page = 'https://finance.yahoo.com/markets/stocks/52-week-gainers/?start=0&count=100'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
        }
    response = requests.get(page, headers=headers)
    soup = BeautifulSoup(response.text, 'html')
    stocks2 = []
    table = soup.find('table', class_="markets-table")
    # print(table)
    # limit = 10
    for row in table.find_all('tr')[1:11]:
        link = row.find('a')  # Finding the link in the row
        if link:
            company_code_span = link.find('span', class_="symbol")
            company_name_span = link.find('span', class_="longName")
            # Extract company code and name
            company_code = company_code_span.text.strip() if company_code_span else "N/A"
            company_name = company_name_span.text.strip() if company_name_span else "N/A"
        cols = row.find_all('td')
        Week_Change = cols[8].text.strip().replace(',', '').replace('%', '')
        Total_Cash = cols[6].text.strip()
            # CEO_Year_Born = cols[4].text.strip()
                        # volume = cols[6].text.strip()
        details1.append({
                "Name": company_name,
                "code": company_code,
                "52-week change": Week_Change,
                "Total cash": Total_Cash,
                        # "CEO_Year_Born": CEO_Year_Born,
                })
        # limit-=1
        # if limit==0:
        #     break
    return details1
def weekchange():
    details1 = scrap_page()
    df = pd.DataFrame(details1)
    df['52-week change'] = pd.to_numeric(df['52-week change'], errors='coerce')
    sorted_df = df.sort_values(by="52-week change", ascending=False)
    top_10 = sorted_df.head(10)
    return top_10
# weekchange()
weekchange()

,Name,code,52-week change,Total cash
0,Summit Therapeutics Inc.,SMMT,1150.00,17.637B
1,"AST SpaceMobile, Inc.",ASTS,598.65,7.133B
2,"Avidity Biosciences, Inc.",RNA,581.11,5.202B
3,"Viking Therapeutics, Inc.",VKTX,467.12,6.862B
4,"ADMA Biologics, Inc.",ADMA,437.33,4.496B
5,"Lumen Technologies, Inc.",LUMN,417.27,7.311B
6,"Janux Therapeutics, Inc.",JANX,387.75,2.534B
7,MicroStrategy Incorporated,MSTR,387.53,31.18B
8,TeraWulf Inc.,WULF,326.19,2.055B
9,"Edgewise Therapeutics, Inc.",EWTX,324.92,2.431B


## 10 largest holds of Blackrock Inc. You can find related info on the Holders tab.
    Blackrock Inc is an investment management corporation.
    Sheet's fields: Name, Code, Shares, Date Reported, % Out, Value.
    All fields except first two should be taken from Holders tab.

In [3]:
import requests
from bs4 import BeautifulSoup


profile_page = f'https://finance.yahoo.com/quote/BLK/holders/'
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
        }
response = requests.get(profile_page, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

#response.text # prints whole context



In [2]:
with open("webpage_content.txt", "w", encoding="utf-8") as file:
    file.write(response.text)

print("HTML content saved to webpage_content.txt")

HTML content saved to webpage_content.txt


In [13]:
import pandas as pd
from bs4 import BeautifulSoup

# Step 1: Read the HTML content from the saved text file
with open("webpage_content.txt", "r", encoding="utf-8") as file:
    html_content = file.read()

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Step 3: Locate the table by its 'data-testid' attribute
table = soup.find('section', {'data-testid': 'holders-top-institutional-holders'}).find('table')

# # Find the table within the section
# table = table_section.find('table')

# Extract the headers from the table
headers = [header.text for header in table.find_all('th')]

# Extract the rows from the table
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    rows.append([col.text.strip() for col in columns])

# Step 4: Convert the data into a pandas DataFrame
df = pd.DataFrame(rows, columns=headers)

# Display the DataFrame
df


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Group Inc,13.22M,"Jun 30, 2024",8.92%,"12,439,142,001"
1,Blackrock Inc.,9.52M,"Jun 30, 2024",6.42%,"8,956,277,644"
2,State Street Corporation,5.94M,"Jun 30, 2024",4.01%,"5,591,683,539"
3,Temasek Holdings (Private) Limited,5.13M,"Jun 30, 2024",3.47%,"4,831,672,332"
4,Bank of America Corporation,5.13M,"Jun 30, 2024",3.47%,"4,831,626,212"
5,Capital Research Global Investors,4.53M,"Jun 30, 2024",3.06%,"4,266,209,597"
6,Morgan Stanley,4.34M,"Jun 30, 2024",2.93%,"4,087,741,098"
7,"Charles Schwab Investment Management, Inc.",3.77M,"Jun 30, 2024",2.54%,"3,546,994,120"
8,Capital World Investors,3.22M,"Jun 30, 2024",2.17%,"3,029,014,555"
9,"Geode Capital Management, LLC",2.79M,"Jun 30, 2024",1.88%,"2,622,638,511"
